# FishBase (cboettig/fishbase) — EDA Notebook

This notebook does an Exploratory Data Analysis (EDA) of the **FishBase Parquet snapshot** hosted on Hugging Face by Carl Boettiger (`cboettig/fishbase`).

## What you'll get
- Inventory of key tables (row counts, distinct `SpecCode`, column counts)
- Schema previews for `cultspec`, `cultsys`, `broodstock`, `diseases`, `diet`, `diet_items`, `aquamaps`, `country`, `comnames`, `biblio`
- Basic data-quality checks (missingness, duplicate keys), and join sanity checks
- Simple charts (matplotlib) to visualize size per table

## Requirements
- Python 3.9+
- `duckdb>=0.10`, `polars>=1.0`, `pandas`, `pyarrow`
- Internet access (to stream Parquet from Hugging Face)

## Notes
- License is **CC-BY-NC** (non-commercial). Cite FishBase & SeaLifeBase accordingly.
- Snapshot used below defaults to **v24.07** (July 2024). You can change `SNAPSHOT`.
- Many tasks require JOINs across tables; prefer `SpecCode`/`StockCode` as keys.


In [3]:
# Setup: imports and DuckDB httpfs extension
import duckdb, polars as pl, pandas as pd, math, textwrap
from pathlib import Path

con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")

# Optional: speed tweaks for remote scanning
con.execute("PRAGMA enable_progress_bar;")
con.execute("PRAGMA threads = 4;")

# Base paths (Hugging Face)
ORG = 'cboettig'
DATASET = 'fishbase'
SNAPSHOT = 'v24.07'  # change to v23.05, etc. if desired
BASE = f"hf://datasets/{ORG}/{DATASET}@main/data/fb/{SNAPSHOT}/parquet"

# Tables of interest (present in v24.07 dir)
TABLES = [
    'cultspec','cultsys','broodstock','diseases','diet','diet_items',
    'aquamaps','country','comnames','biblio','brains','alieninvasive',
    'cigua','ciguafb','bruvsspecies','bruvssample','countecosystem','countfao'
]

def parquet_path(tbl: str) -> str:
    return f"{BASE}/{tbl}.parquet"

def head_df(tbl: str, n: int = 5) -> pd.DataFrame:
    q = f"SELECT * FROM read_parquet('{parquet_path(tbl)}') LIMIT {n}"
    return con.execute(q).df()

def schema_df(tbl: str) -> pd.DataFrame:
    q = f"PRAGMA parquet_schema('{parquet_path(tbl)}')"
    return con.execute(q).df()

def table_counts(tbl: str) -> dict:
    # count rows, distinct SpecCode (if exists), and columns
    path = parquet_path(tbl)
    cols = con.execute(f"SELECT * FROM read_parquet('{path}') LIMIT 0").df().columns
    rowcount = con.execute(f"SELECT count(*) AS n FROM read_parquet('{path}')").fetchone()[0]
    has_spec = 'SpecCode' in cols
    nspec = None
    if has_spec:
        nspec = con.execute(f"SELECT count(DISTINCT SpecCode) FROM read_parquet('{path}')").fetchone()[0]
    return {
        'table': tbl,
        'n_rows': rowcount,
        'n_cols': len(cols),
        'n_distinct_SpecCode': nspec,
    }

print('Ready. BASE =', BASE)


Ready. BASE = hf://datasets/cboettig/fishbase@main/data/fb/v24.07/parquet


In [4]:
# Inventory summary for all tables in TABLES (may take a few minutes)
summaries = []
for tbl in TABLES:
    try:
        summaries.append(table_counts(tbl))
    except Exception as e:
        summaries.append({'table': tbl, 'error': str(e)})
summary_df = pd.DataFrame(summaries).sort_values('n_rows', ascending=False)
summary_df


,table,n_rows,n_cols,n_distinct_SpecCode
9,biblio,723690,19,35729.0
8,comnames,330105,35,29751.0
17,countfao,202546,7,35027.0
7,country,194912,36,35071.0
16,countecosystem,112544,6,NaN
5,diet_items,41364,15,NaN
6,aquamaps,33518,7,NaN
14,bruvsspecies,14546,30,NaN
4,diet,7104,40,NaN
15,bruvssample,6865,18,NaN


In [5]:
# Save summary to CSV
outdir = Path('eda_outputs'); outdir.mkdir(exist_ok=True)
summary_path = outdir / 'fishbase_table_inventory.csv'
summary_df.to_csv(summary_path, index=False)
summary_path


WindowsPath('eda_outputs/fishbase_table_inventory.csv')

In [6]:
# Preview schemas of key tables
KEYS = ['cultspec','cultsys','broodstock','diseases','diet','diet_items','aquamaps','country','comnames','biblio']
schemas = {k: schema_df(k) for k in KEYS}
for k, df in schemas.items():
    print(f"\n=== Schema: {k} ===")
    display(df)


CatalogException: Catalog Error: Pragma Function with name "parquet_schema" is not in the catalog, but it exists in the parquet extension.

In [ ]:
# Quick head previews
for k in ['cultspec','cultsys','diseases','diet','aquamaps','country','comnames']:
    print(f"\n=== Head: {k} ===")
    display(head_df(k))


## Data quality checks
We compute missingness and duplicates for a few key fields, then check a simple join path.


In [ ]:
# Missingness on selected tables/fields
CHECKS = {
    'cultsys': ['SpecCode','Country','System','Salinity','Locality'],
    'cultspec': ['SpecCode','Country','Status','Salinity'],
    'diseases': ['SpecCode','DiseaseI','Agenttype','Treatment','Country'],
    'diet': ['SpecCode','DietTroph','FoodIII'],
}

def missingness(tbl: str, cols: list[str]) -> pd.DataFrame:
    df = con.execute(f"SELECT * FROM read_parquet('{parquet_path(tbl)}')").df()
    out = []
    for c in cols:
        if c in df.columns:
            nmiss = int(df[c].isna().sum())
            out.append({'table': tbl, 'column': c, 'n_missing': nmiss, 'pct_missing': nmiss/len(df) if len(df)>0 else math.nan})
    return pd.DataFrame(out)

miss_list = [missingness(t, cs) for t, cs in CHECKS.items()]
miss_df = pd.concat(miss_list, ignore_index=True)
miss_df.sort_values(['table','pct_missing'], ascending=[True, False])


In [ ]:
# Duplicate key checks
dups = {}
dups['cultsys'] = con.execute(
    f"""
    WITH t AS (
      SELECT SpecCode, Country, System, COUNT(*) AS n
      FROM read_parquet('{parquet_path('cultsys')}')
      GROUP BY 1,2,3
    ) SELECT * FROM t WHERE n>1
    """
).df()

dups['cultspec'] = con.execute(
    f"""
    WITH t AS (
      SELECT SpecCode, Country, Status, COUNT(*) AS n
      FROM read_parquet('{parquet_path('cultspec')}')
      GROUP BY 1,2,3
    ) SELECT * FROM t WHERE n>1
    """
).df()

for k, df in dups.items():
    print(f"\n=== Duplicates in {k} ===")
    display(df.head())


In [ ]:
# Simple join sanity: cultsys x country on (SpecCode, Country)
joined = con.execute(
    f"""
    SELECT cs.SpecCode, cs.Country, COUNT(*) AS n_cultsys, COUNT(DISTINCT cy.Status) AS n_country_status
    FROM read_parquet('{parquet_path('cultsys')}') cs
    LEFT JOIN read_parquet('{parquet_path('country')}') cy
      ON cs.SpecCode = cy.SpecCode AND cs.Country = cy.Country
    GROUP BY 1,2
    ORDER BY n_country_status DESC
    LIMIT 20
    """
).df()
joined.head()


## Visualizations
Bar chart of table sizes and a few categorical breakdowns.


In [ ]:
import matplotlib.pyplot as plt

# Plot table sizes
df_plot = summary_df.dropna(subset=['n_rows']).nlargest(15, 'n_rows')
plt.figure()
plt.barh(df_plot['table'], df_plot['n_rows'])
plt.xlabel('Rows')
plt.ylabel('Table')
plt.title('Top tables by row count')
plt.gca().invert_yaxis()
plt.show()

# Breakdown: cultsys by System (if column exists)
try:
    df_cultsys = con.execute(f"SELECT System, COUNT(*) n FROM read_parquet('{parquet_path('cultsys')}') GROUP BY 1 ORDER BY n DESC").df()
    plt.figure()
    plt.barh(df_cultsys['System'].astype(str), df_cultsys['n'])
    plt.xlabel('Rows')
    plt.ylabel('System')
    plt.title('cultsys — rows by System')
    plt.gca().invert_yaxis()
    plt.show()
except Exception as e:
    print('cultsys breakdown skipped:', e)


## Next steps
- Add province-level spatial overlay (AquaMaps grid → GADM provinces)
- Build a tidy **SpecCode–Country–System** panel for downstream LTR modeling
- Persist intermediate tables locally (Parquet) for faster iteration
